# About: Status Report of running VMs

---

現在起動中のVMの一覧を確認し、各VMにpingが到達するかをチェックするNotebook。

## *Operation Note*

*This is a cell for your own recording.  ここに経緯を記述*

# Notebookと環境のBinding

Inventory中のgroup名でBind対象を指示する。**Bind対象はKVMがインストールされた物理マシンとする。**

In [1]:
target_group = 'test-hypervisor'

Bind対象への疎通状態を確認する。SUCCESSと表示されればよい。

In [2]:
!ansible -m ping {target_group}

XXX.XXX.XXX.105 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


# VMの一覧確認

起動中のVMの一覧を確認する。

In [3]:
!ansible -b -a 'virsh list' {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
 Id    Name                           State
----------------------------------------------------
 5     testvm-001                     running



各VMのIPアドレスを確認する。

In [4]:
vmnames = !ansible -b -a 'virsh list' {target_group}
vmnames = vmnames[1:]
vmnames = vmnames[map(lambda l: l[0], filter(lambda l: l[1].startswith('----'), enumerate(vmnames)))[0] + 1:]
vmnames = map(lambda l: l.split()[1], vmnames)
vmnames

['testvm-001']

VM名-IPアドレス のtupleのリストを作成する。

In [5]:
import re
def get_mac_address(vmname):
    domiflist_stdio = !ansible -b -a "virsh domiflist {vmname}" {target_group}
    mac_pattern = re.compile(r'.*bridge.*\s([0-9a-f\:]+)\s*')
    vmmac = [mac_pattern.match(line).group(1) for line in domiflist_stdio if mac_pattern.match(line)][0]
    return vmmac

def get_ip_address(vmmac):
    leases_stdio = !ansible -b -a "grep {vmmac} /var/lib/dnsmasq/dnsmasq.leases" {target_group}
    ip_pattern = re.compile(r'.*\s([0-9a-f\:]+)\s+([0-9\.]+)\s.*')
    ipaddr = [ip_pattern.match(line).group(2) for line in leases_stdio if ip_pattern.match(line)][0]
    return ipaddr

In [6]:
vmdescs = zip(vmnames, map(lambda mac: get_ip_address(mac), map(lambda n: get_mac_address(n), vmnames)))
vmdescs

[('testvm-001', 'XXX.XXX.XXX.66')]

# VMの到達性確認

VMへの到達性を確認する。

In [7]:
for name, ipaddr in vmdescs:
    print('VM: {0} IP address: {1}'.format(name, ipaddr))
    !ping -c 4 {ipaddr}
    print('\n')

VM: testvm-001 IP address: XXX.XXX.XXX.66
PING XXX.XXX.XXX.66 (XXX.XXX.XXX.66) 56(84) bytes of data.
64 bytes from XXX.XXX.XXX.66: icmp_seq=1 ttl=63 time=0.577 ms
64 bytes from XXX.XXX.XXX.66: icmp_seq=2 ttl=63 time=0.450 ms
64 bytes from XXX.XXX.XXX.66: icmp_seq=3 ttl=63 time=0.436 ms
64 bytes from XXX.XXX.XXX.66: icmp_seq=4 ttl=63 time=0.436 ms

--- XXX.XXX.XXX.66 ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 2998ms
rtt min/avg/max/mdev = 0.436/0.474/0.577/0.065 ms


